In [1]:
"""
Calculate fire severity statistics within AFD observations from MODIS and VIIRS.
Severity index: Composite Burn Severity (CBI), calculated in GEE via Parks (2018)

Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [2]:
# Load the gridded FRP data
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_gridstats')
print(f"Number of grid cells: {grid.size().getInfo()}")
grid = grid.select(['grid_index'])
print(grid.first().propertyNames().getInfo())

Number of grid cells: 55936
['system:index', 'grid_index']


In [3]:
# Load topographic collections

# USGS NED 10-m DEM
dem = ee.Image("USGS/3DEP/10m")
slope = ee.Terrain.slope(dem) # calculate slope from DEM
dem = dem.rename('elev') # rename the elevation band

# Topographic Position Index (TPI) (USGS NED 10m-based)
tpi = ee.Image("CSP/ERGo/1_0/US/mTPI").rename('tpi')

# Continuous Heat-Insolation Load Index (CHILI)
chili = ee.Image("CSP/ERGo/1_0/US/CHILI").rename('chili')

# combine topographic variables
topo = dem.addBands([slope,tpi,chili])
print(topo.bandNames().getInfo())

['elev', 'slope', 'tpi', 'chili']


In [4]:
# Run the reduction on the grid
stats = topo.reduceRegions(
    collection=grid,
    reducer=ee.Reducer.mean()
)
stats.limit(10)

In [5]:
# export to drive
export_task = ee.batch.Export.table.toDrive(
    collection=stats,
    description='gridstats_topo',
    fileNamePrefix='gridstats_topo',
    fileFormat='CSV', 
    folder='Topo'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 30) 

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!
